Status: ✅ done

## Exercise 9

---

The weekend is over and we are back to business of machine learning, and specifically for this exercise we are going to focus on `generative models`. (by know you should what these are, if not, I suggest you revisit the previous exercise)

> Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import multivariate_normal

from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

### LDA & QDA overview

---

> Quick recap on generative models

As discussed in the previous exercise, generative models work from a high level as follows:

1. From the provided training data, try to estimate $p(C_k)$ `class prior` and then also $p(x|C_k)$ `class conditionals`. This corresponds to training part of the ML process.
2. Then use these estimates to compute `posterior probability` using `Bayes theorem` (part of prediction process):
    
$
p(C_k |x) = \frac{p(x|C_k)p(C_k)}{p(x)}
$

where $p(x) = \sum_l^K p(x|C_l)p(C_l)$. Note that since this term is common to all classes, what really matters are the terms in numerator, i.e., class **priors** and **conditionals**. So in other words, we could only consider the `joint probability` $p(x, y)$ since:

$
p(C_k, x) = p(x|C_k)p(C_k)
$
    
3. Finally, use some decision rule $d(x)$ to predict a class for given sample based on computed posterior probabilites class of the given sample $x$. As discussed in the last exercise, the most `optimal decision under 0-1 loss is to choose the class with highest posterior probability` - `Bayes Classifier`. So our decision is then:
    
$
d(x) = \text{argmax } p(C_k|x)
$

Note that you could also use:

$
d(x) = \log(\text{argmax } p(C_k|x))
$
    
Since the argmax will remain the same.

> Getting the decision function from Bayes theorem (advanced)

In this section, my goal will be to show how we get to the decision functions $d(x)$ for both classifiers. Note that this is quite cumbersome process so in the exam you will not be asked to reproduce this, but you should be able to explain the high level strategy how we get to the respective decision functions:
- we start with `Bayes` theorem
- we then plug in corresponding approximations for `class conditionals` and `priors`

Therefore, the below derivation is only for those eager ones who really want to understand things in depth 🤔 (but there are more important things to put your focus on)

Let me start by writing down the Bayes theorem:

$
p(C_k|x) = \frac{P(x|C_k)p(C_k)}{p(x)}
$

Since $p(x)$ serves as a normalisation and is common to all classes $K$, we can simply ignore it and reduce it to (note that we can only do this because of what I just mentioned):

$
p(C_k|x) \propto P(x|C_k)p(C_k)
$

Since we want to choose a class with highest posterior probability, we can use log transformation on this formula and we will still get the same result:

$
\log (p(C_k|x)) \propto \log (P(x|C_k)p(C_k))
$

We now continue by plugging our estimates, for our class prior $p(C_k)$ we have:

$
p(C_k) = \pi_k = \frac{n_k}{n}
$

And for our class conditional $p(x|C_k)$ we have:

$
p(x|C_k) = \frac{1}{2\pi^{\frac{m}{2}}|\Sigma_k|^{-\frac{1}{2}}}\exp(-\frac{1}{2}(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k))
$

So overall we have:

$
\log (p(C_k|x)) \propto \log(\frac{1}{2\pi^{\frac{m}{2}}|\Sigma_k|^{-\frac{1}{2}}}\exp(-\frac{1}{2}(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k))\pi_k)
$

We recall the rule for division in our log term, i.e.:

$
\log \frac{x}{y} = \log x - \log y
$

And similarly for product where we instead use plus. First, using the division rule we obtain:

$
\log{(p(C_k|x))} \propto \log(\exp(-\frac{1}{2}(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k))\pi_k) - \log(2\pi^{\frac{m}{2}}|\Sigma_k|^{-\frac{1}{2}})
$

And now using multiplication rule we can write:

$
\log(P(C_k|x)) \propto -\frac{1}{2}(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k) + \log(\pi_k) + \frac{1}{2}\log(|\Sigma_k|)- \frac{m}{2}\log(2\pi)
$

To make it nicer we multiply it by two:

$
2\log(P(C_k|x)) \propto -(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k) + 2\log(\pi_k) + \log(|\Sigma_k|)- m\log(2\pi)
$

Notice that the last term does not depend on $k$, therefore we can drop it and obtain:

$
2\log(P(C_k|x)) \propto -(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k) + 2\log(\pi_k) + \log(|\Sigma_k|)
$

We now focus on the expansion of the first term:

$
-(x - \mu_k)^T\Sigma_k^{-1}(x - \mu_k)
$

We first use the following linear algebra rule:

$
(A - B)^T = A^T - B^T
$

Therefore, we get:

$
-(x^T - \mu_k^T)\Sigma_k^{-1}(x - \mu_k)
$

Then we expand the left bracket as follows:

$
-(x^T\Sigma_k^{-1} - \mu_k^T\Sigma_k^{-1})(x - \mu_k)
$

Now we again expand the two brackets as:

$
-(x^T\Sigma_k^{-1}x - \mu_k^T\Sigma_k^{-1}x - x^T\Sigma_k^{-1}\mu_k + \mu_k^T\Sigma_k^{-1}\mu_k)
$

We then use another rule for working with matrices:

$
(AB)^T = B^TA^T
$

In addition, realize that $\Sigma_k^{-1}$ is a **square symmetrical matrix**. This means if we take its transpose, we again obtain the exact same matrix. One more note, each of the terms above ends up being a real number, and we know that transpose of a real number yields again the exact same number.  Given all these assumptions, we can write the following:

$
x^T\Sigma_k^{-1}\mu_k = (x^T\Sigma_k^{-1}\mu_k)^T = (\Sigma_k^{-1}\mu_k)^Tx =  \mu_k^T\Sigma_k^{-1}x
$

Therefore, we can reduce the two middle terms into one as follows:

$
-(x^T\Sigma_k^{-1}x - 2\mu_k^T\Sigma_k^{-1}x + \mu_k^T\Sigma_k^{-1}\mu_k)
$

So overall we obtain:

$
2\log(P(C_k|x)) \propto -(x^T\Sigma_k^{-1}x - 2\mu_k^T\Sigma_k^{-1}x + \mu_k^T\Sigma_k^{-1}\mu_k) + 2\log(\pi_k) + \log(|\Sigma_k|)
$

We can then do the following substitution:

- $a = -\mu_k^T\Sigma_k^{-1}\mu_k + 2\log(\pi_k) + \log(|\Sigma_k|)$ (last three terms)
- $b = 2\mu_k^T\Sigma_k^{-1}$
- $c = -\Sigma_k^{-1}$

So we obtain:

$
2\log(P(C_k|x)) \propto x^Tcx + bx + a = cx^Tx + bx + a
$

Due to the first term, we have the **quadratic** discriminant. Notice the **similarity to LDA**, where the only difference is that we have $\Sigma$ instead of $\Sigma_k$. Therefore we drop the first quadratic term as it is no longer dependent on $k$. As such for LDA we can write:

$
2\log(P(C_k|x)) \propto bx + a
$

So this means we can conclude the following for each respective classifier:

- `LDA`: $d(x) = \text{argmax}_k \text{ } bx + a$
- `QDA`: $d(x) = \text{argmax}_k \text{ } cx^Tx + bx + a$

In simple words, you predict the class with the highest result value $r$ where $r \propto P(C_k | x)$.

> Bonus: LDA and Mahalanobis distance

So now we have arrived at the formula for $d(x)$ of `LDA`. If we rewrite the above derived formula in a more compact form, we get:

$
\text{argmax}_k \text{ } d(C_k| x)_{\mu_k, \Sigma} = 2\log \pi_k - (x-\mu_k)^T\Sigma^{-1}(x-\mu_k)
$

This means that the decision essentially depends on two terms. The first term is simple, it is the log of prior of the given class. Recall that prior can be between 0 and 1, therefore:
- large prior yields **small negative number**
- small prior yields **large negative number**

The second term can be interpretted as a **square of special kind of distance**. (I will explain in a bit what distance and why square) Since distance can not be negative, we will get some positive value miltiplied by negative sign. Therefore, the smaller this distance is, the better. Why? Let me give you an example.

> We get **high** class prior = **log negative number**, we get **small** distance = **low negative number**, result must be also some **low negative** number. Ideally as close as possible to zero (best possible result). This is because we use **argmax**.

Now you hopefully got the big picture. Time to explain the distance metric. It is called **Mahalanobis distance** and is defined as follows:

$
d_m = \sqrt{(x-\mu_k)^T\Sigma^{-1}(x-\mu_k)}
$

Therefore, if we use square transformation we get:

$
d_m^2 = (x-\mu_k)^T\Sigma^{-1}(x-\mu_k)
$

Plugging this back into our formula:

$
\text{argmax}_k \text{ } d(C_k| x)_{\mu_k, \Sigma} = 2\log \pi_k - d_m^2
$

So what is so special about `Mahalanobis distance` and why can not we simply use the `Eucliden distance`? Simple answer would be:

> Eucliden distance measures **distance between two points**, whereas Mahalanobis distance measures **distance between a point and given distribution**.

In other words, Mahalanobis distance takes into account variance of each feature as well as dependence of features (correlation). In practice, the distribution to which we are measuring the distance is the **class conditional gaussian distribution**. Therefore, the closer the given input $x$ is to the mean of the given gaussian, the better. This makes sense since we want to classify the given $x$ to the group (class) to which it looks most similar to in the feature space. (the similarity here is measure by Mahalanobis distance) If you want even more details, then I suggest you read this [article](https://www.machinelearningplus.com/statistics/mahalanobis-distance/).

> Section summary

This section was supposed to summarize core things about both LDA and QDA as well as show some more advanced things like Mahalanobis distance. In the below section, we will practice these theoretical concepts.

### Deep dive into LDA and decision theory

---

Let's consider the following `population` defined by its class conditionals:

$
\begin{aligned}
p(x \mid Y=\text { black }) &=\mathcal{N}(2,1) \\
p(x \mid Y=\text { red }) &=\mathcal{N}(4,1) \\
p(x \mid Y=\text { blue }) &=\mathcal{N}(7,1)
\end{aligned}
$

NB! `same variance for all classes`

and the class priors $\left(\pi_{\mathrm{black}}, \pi_{\mathrm{red}}, \pi_{\mathrm{blue}}\right)=(0.6,0.1,0.3)$. Here is also a corresponding plot of unormalized posteriors $P(C_k | X)$ for each $k$:

![population plot](images/img1.png)

> Solving the problem in theory

Before jumping into code, let's start with simple evaluation of the problem. First, how would the decision regions look like? We can draw these as follows:

![Decision regions](images/img2.png)

Now, how many possible misclassifications can we make and what would be the corresponding probabilites? Since we have 3 classes, the confusion matrix would be 3 x 3. Using this notion we can visualize it as follows:

![Misclassifications](images/img3.png)

Finally, let's actually find out the exact boundaries for the decision regions. Since we have an access to the image, we can easily see that we are looking for value $x$ such that $P(C_{black}|x) = P(C_{red}|x)$ for the first boundary and similarly for the second one. To simplify things, we will only use function $g(x)$ which is proportional to the posterior probability:

$
d(x) = \text{argmax}_k \text{ } g(x) = \text{argmax}_k \text{ } bx + a
$

If we now substitute for the parameters $a, b$, we get: `Important Formula for Decision Boundary`

$
g_{k}(x) = \frac{\mu_k}{\sigma_k^{2}}x - \frac{\mu_k^{2}}{2\sigma_k^{2}} + log(\pi_k)
$

Therefore, for the first boundary, we can start by writing:
$
g_{black}(x) = g_{red}(x)
$

This yields the following linear equations:

$
\begin{aligned}
g_{black}(x) \approx 2x - 2.5 \\
g_{red}(x) \approx 4x - 10.3
\end{aligned}
$

Then we solve for $x$ as follows:

$
\begin{aligned}
g_{black}(x) = g_{red}(x) \\
2x - 2.5 = 4x - 10.3 \\
-2x = - 7.8 \\
x = 3.9
\end{aligned}
$

For the second boundary, we can proceed similarly and write:

$
\begin{aligned}
g_{red}(x) \approx 4x - 10.3 \\
g_{blue}(x) \approx 7x - 25.7
\end{aligned}
$

And then again solve for $x$:

$
\begin{aligned}
g_{red}(x) = g_{blue}(x) \\
4x - 10.3 =  7x  - 25.7 \\
-3x = -15.4 \\
x = 5.1
\end{aligned}
$

Therefore we can conlude that the two boundaries are at $x_1 = 3.9$ and $x_2 = 5.1$.

> Solving the problem in practice

First, we want to define our `Bayes` classifier based on the provided information:

In [3]:
class BayesClassifier:

    """Bayes Classifier based on the provided information
    """

    def __init__(self):
        # -- class conditionals
        self.params = [(2, 1), (4, 1), (7, 1)]

        # -- class priors
        self.priors = (.6, .3, .1)
    
    def predict_probs(self, x):
        """Predict posteriors for given input

        Attributes
        ----------
        x : 1d array
            Each element represents one possible input since we have one dimensional distribution

        Returns
        -------
        result : 2d array
            n x m matrix where n is the number of inputs and k is the number of classes
        
        Todo
        ----
        I am converting back and forth list to arrays which is not so good for performance but it works 
        for educatinal purposes. Feel free to rewrite this more efficiently.
        """

        # Result will be n, m matrix
        n, k = x.shape[0], 3

        # Result for now just 2d list
        result = []

        for i in range(n):

            # Compute the likelihhods
            likelihoods = []
            xin = x[i]
            for j in range(k):
                likelihood = stats.norm.pdf(xin, self.params[j][0], self.params[j][1])*self.priors[j]
                likelihoods.append(likelihood)
            
            # Get posteriors
            px = sum(likelihoods)
            posteriors = list(np.array(likelihoods)/px)

            # Save the result
            result.append(posteriors)
        
        return np.array(result)

Let's test it out:

In [4]:
# Some data
x = np.arange(2, 8, .1)

# Get the predictions for each input
bayes = BayesClassifier()
posteriors = bayes.predict_probs(x)

# Show first 5 rows
posteriors[:5]

array([[9.36620517e-01, 6.33789015e-02, 5.81743303e-07],
       [9.23659140e-01, 7.63399144e-02, 9.45859649e-07],
       [9.08306506e-01, 9.16919604e-02, 1.53353832e-06],
       [8.90233129e-01, 1.09764393e-01, 2.47806789e-06],
       [8.69110487e-01, 1.30885524e-01, 3.98870282e-06]])

Let's also double check that each row sums to one:

In [5]:
posteriors.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

Now, back to the problem of getting the probabilities, let me just reinsert the image here so we know what we are up to:

![misclassification](images/img3.png)



We are going to solve the problem in three stages. We are going to get the misclassification probabilities highlighted by

(1) The `green` color: **we predict black, but true is either red or blue**. 

In [23]:
# Side note: in the posterior matrix
#   Each row = posteriors for the given sample
#   Columns represent posteriors for: Black (0), Red (1), Blue (2)

# Stage 1
# -- Get the posteriors
x1 = np.arange(-3, 3.9, .01)
posteriors1 = bayes.predict_probs(x1)

# Compute each respective error
true_red_predict_black = posteriors1[:, 1].sum()
true_blue_predict_black = posteriors1[:, 2].sum()
green_area = true_red_predict_black + true_blue_predict_black
green_area

69.26808000994428

(2) The `blue` color: **we predict red, but true is either black or blue**. 

In [24]:
# Side note: in the posterior matrix
#   Each row = posteriors for the given sample
#   Columns represent posteriors for: Black (0), Red (1), Blue (2)

# Stage 2
# -- Get the posteriors
x2 = np.arange(3.9, 5.1, .01)
posteriors2 = bayes.predict_probs(x2)

# Compute each respective error
true_black_predict_red = posteriors2[:, 0].sum()
true_blue_predict_red = posteriors2[:, 2].sum()
blue_area = true_black_predict_red + true_blue_predict_red
blue_area

15.620017797080765

(3) The `yellow` color: **we predict blue, but true is either black or red**. 

In [25]:
# Side note: in the posterior matrix
#   Each row = posteriors for the given sample
#   Columns represent posteriors for: Black (0), Red (1), Blue (2)

# Stage 3
# -- Get the posteriors
x3 = np.arange(5.1, 11, .01)
posteriors3 = bayes.predict_probs(x3)

# Compute each respective error
true_black_predict_blue = posteriors3[:, 0].sum()
true_red_predict_blue = posteriors3[:, 1].sum()
yellow_area = true_black_predict_blue + true_red_predict_blue
yellow_area

80.49798357147995

Let's double check that we got correct results:

In [9]:
true_red_predict_blue > true_black_predict_blue

True

The reason why this should be true is that if we look at the graph, then the red line has way higher posteriors than the black line so naturally, if we sum all over these, black sum should be lower. Finally, we can also obtain the total irreducible bayes error rate as:

In [10]:
total_error = green_area + blue_area + yellow_area
n1, n2, n3 = len(x1), len(x2), len(x3)
bayes_error = total_error/(n1 + n2 + n3)
bayes_error

0.11813291527036071

Note that this number is just an approximation since the input $x$ is not continuous (the step size is $0.01$) so depending on that you might get a slightly different value.

> Estimating parameters and comparing this to the Bayes classifier

Now, we are going to sample from the defined population `training` and `test` data, train a new `LDA` model and see how it performs against the above `Bayes` classifier. Let's start with sampling:

In [11]:
# Define size of of sample
N = 1400

# Save the sampled data to these
x = []
y = []

for _ in range(N):
    p = np.random.random()
    if p < .6:
        p = np.random.random()
        x.append(stats.norm.ppf(p, 2, 1))
        y.append(0)
    elif p < .9:
        p = np.random.random()
        x.append(stats.norm.ppf(p, 4, 1))
        y.append(1)
    else:
        p = np.random.random()
        x.append(stats.norm.ppf(p, 7, 1))
        y.append(2)

# Save it as pandas
d = {'x': x, 'y': y}
data = pd.DataFrame(data=d)

In [12]:
data.head()

,x,y
0,2.242353,0
1,1.086894,0
2,4.208785,1
3,4.130867,1
4,4.662471,1


Let's do train test split:

In [13]:
train, test = train_test_split(data, test_size=0.3)

Now, we can fit our `LDA` model:

In [14]:
m1 = LinearDiscriminantAnalysis().fit(train[['x']], train['y'])

If you wanted to do this manually, you need to estimate class means, and variance:

In [15]:
# Class means
mean_black = train[train['y'] == 0]['x'].mean()
mean_red = train[train['y'] == 1]['x'].mean()
mean_blue = train[train['y'] == 2]['x'].mean()

# Variance
common_var = train['x'].var()

Then to get predictions for new data, you would write the exact same code as I wrote for the `bayes classifier`. Feel free to implement your own `LDA`, but I am going to use `sklearn` for this since the main focus is on the interpretation of the results. There, let's now see how our model works on test data:

In [26]:
yhat = m1.predict(test[['x']])
cm = confusion_matrix(test['y'], yhat)
cm

array([[225,  26,   0],
       [ 35,  88,   2],
       [  0,   7,  37]], dtype=int64)

Not that bad... 😌 Let's now divide the misclassifications by the total number of misclassifications:

In [17]:
misclassifications_total = cm.sum() - np.diagonal(cm).sum()
cm_norm = cm/misclassifications_total
cm_norm

array([[3.21428571, 0.37142857, 0.        ],
       [0.5       , 1.25714286, 0.02857143],
       [0.        , 0.1       , 0.52857143]])

First, ignore the digaonal values since these do not mean anything. What we are interested however are the non-digaonal values. For instance, we see that the biggest misclassification rate has `true is red, but predicted black`:

In [18]:
cm_norm[1, 0]

0.5

Let's see what was the rate in `Bayes` classifier:

In [19]:
true_red_predict_black/total_error

0.4184978033242

Relatively similar. How about `true is black, predicted blue`?

In [20]:
true_black_predict_blue/total_error 

0.006099309430870282

 Finally, let's check the estimate of bayes error rate:

In [21]:
1 - accuracy_score(test['y'], yhat)

0.16666666666666663

Given that our model is supposed to estimate `Bayes`, we can conclude that it is doing that relatively well.
One last thought experiment, what if we trained the model on a sample with balanced classes, would we get even closer to estimating `bayes classifier`? As I have mentioned in the previous exercise, ideally we want our model to see as many as possible different inputs for each respective class. Therefore, if we would change the priors in training to do this, I believe it would have a positive impact on our model's performance as long as we then re-adjust the posteriors according to the actual priors. (See previous exercise where there is a whole section dedicated to it)

> Section summary

In this section, we have focused on working with `LDA` in connection to the decision theory. We first saw how we would make predictions if we know the whole population's parameters. Then we switched perspective and focused on how estimating these parameters and then comparing our model based on these parameters to the `Bayes` classifier which is based on the population's parameters. This showed that despite having access to all information, there will always be some space for error which comes from the uncertainity when predicting given class with highest posterior probability.

### LDA vs QDA

---

> Theory

The core difference between `LDA` and `QDA` is the following assumption:

> QDA assumes each class has a specific mean as well as variance

You can check this assumption by plotting the data. Alternatively, you can fit both models and compare their generalization error. In general, `QDA` is more complex, which you might need or not. This also means that you need to estimate more parameters for `QDA` and as such you need more data. To conclude, whether to use `QDA` or `LDA` depends on the data at hand: is it aligned with either model's assumption? how much data do you actually have?

> Practice

Let's try this in practice - we will QDA and see its generalization error.

In [22]:
m2 = QuadraticDiscriminantAnalysis().fit(train[['x']], train['y'])
1 - accuracy_score(test['y'], m2.predict(test[['x']]))

0.17142857142857137

Almos the same performance, therefore the conclusion is for this particualr case that `LDA` is good enough since we also saw we got very close to the `Bayes` classifier. Session concluded! 🥳

---

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dac56272-d26a-4895-8563-759c36ec57fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>